In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import sklearn

from pandas.io.data import DataReader
from sklearn.linear_model import LogisticRegression
from sklearn.lda import LDA
from sklearn.qda import QDA
from keras.models import Sequential
from keras.layers import LSTM, Dense
import random
from sklearn import datasets, linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from itertools import product
from sklearn.ensemble import VotingClassifier
from sklearn import preprocessing
from sklearn import cross_validation
from scipy.stats import spearmanr 
import xgboost as xgb
from sklearn import cross_validation
from sklearn.metrics import accuracy_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/io/data.py:35: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 a

In [3]:
df = pd.read_csv('../output/dat_prediction.csv', index_col=0)

In [4]:
def dataselection(model, m=4):
    model = model.iloc[:,1:]
    
    X = np.array(model)[:,1:] 
    Y = np.array(model)[:,0] 
    
    n = len(model)
    
    X_test = X[:(n + 1) /m]
    Y_test = Y[:(n + 1) /m]
    X_train = X[(n + 1) /m:]
    Y_train = Y[(n + 1) /m:]
    return X_train, Y_train, X_test, Y_test, X, Y

X_train, Y_train, X_test, Y_test, X, Y = dataselection(df, m=4)

def classificationevaluation(Y_test, Y_pred):
    print '======================================'
    n = len(Y_test)
    print 'ar is: ', accuracy_score(list(Y_test), list(Y_pred))
    print 'er is: ', 1-accuracy_score(list(Y_test), list(Y_pred))
    diff = list(Y_test - Y_pred)

    print 'Type I error rate: ',  diff.count(1)/(n+0.0)
    print 'Type II error rate: ', diff.count(-1)/(n+0.0)

In [7]:
#RF
def randomforest():
    print "**************************************"
    tuned_parameters = {"max_depth": [7,9,11, None],
              "max_features": ['auto','sqrt','log2',None]}
    
    clf = RandomForestClassifier(n_estimators = 100)
    
    gs = GridSearchCV(clf, tuned_parameters, cv=5)
    gs.fit(X_train, Y_train)

    print("Best parameters set found on development set:")
    print(gs.best_params_)

    clf = RandomForestClassifier(**gs.best_params_)
    clfboost = AdaBoostClassifier(clf, n_estimators=50, learning_rate=1.0)
    clfboost.fit(X_train, Y_train)
    
    clf = clf.fit(X_train, Y_train)
    y_pred = clf.predict(X_test)
    print '======================================'
    print 'Random Forest'
    classificationevaluation(Y_test, y_pred)
    print '======================================'
    print 'adaboost'
    classificationevaluation(Y_test, clfboost.predict(X_test))

In [8]:
randomforest()

**************************************
Best parameters set found on development set:
{'max_features': 'sqrt', 'max_depth': 7}
Random Forest
ar is:  0.519114688129
er is:  0.480885311871
Type I error rate:  0.154929577465
Type II error rate:  0.325955734406
adaboost
ar is:  0.51106639839
er is:  0.48893360161
Type I error rate:  0.189134808853
Type II error rate:  0.299798792757


In [9]:
#NB
def NB():
    print "**************************************"
    clf = GaussianNB()
    AdaBoostClassifier(clf, n_estimators=50, learning_rate=1.0)
                            
    clf = clf.fit(X_train, Y_train)
    clfboost = AdaBoostClassifier(clf, n_estimators=50, learning_rate=1.0)
    clfboost.fit(X_train, Y_train)
    
    y_pred = clf.predict(X_test)
    print '======================================'
    print 'NB'
    
    classificationevaluation(Y_test, y_pred)
    print '======================================'
    print 'adaboost'
    classificationevaluation(Y_test, clfboost.predict(X_test))

In [10]:
NB()

**************************************
NB
ar is:  0.53722334004
er is:  0.46277665996
Type I error rate:  0.199195171026
Type II error rate:  0.263581488934
adaboost
ar is:  0.51106639839
er is:  0.48893360161
Type I error rate:  0.0764587525151
Type II error rate:  0.412474849095


In [11]:
#XGBoosting
def xgboostresult():
    print "**************************************"
    tuned_parameters = {"max_depth": [3,7,9,11],
                        'n_estimators': [100,200]}

    clf = xgb.XGBClassifier()
    gs = GridSearchCV(clf, tuned_parameters, cv=5)
    gs.fit(X_train, Y_train)

    print("Best parameters set found on development set:")
    print(gs.best_params_)

    clf = xgb.XGBClassifier(**gs.best_params_)
    clfboost = AdaBoostClassifier(clf, n_estimators=50, learning_rate=1.0)
    clfboost.fit(X_train, Y_train)
    
    clf = clf.fit(X_train, Y_train)
    y_pred = clf.predict(X_test)
    print '======================================'
    print 'XGBoost'
    classificationevaluation(Y_test, y_pred)
    print '======================================'
    print 'adaboost'
    classificationevaluation(Y_test, clfboost.predict(X_test))

In [12]:
xgboostresult()

**************************************
Best parameters set found on development set:
{'n_estimators': 100, 'max_depth': 3}
XGBoost
ar is:  0.494969818913
er is:  0.505030181087
Type I error rate:  0.189134808853
Type II error rate:  0.315895372233
adaboost
ar is:  0.46277665996
er is:  0.53722334004
Type I error rate:  0.53722334004
Type II error rate:  0.0


In [13]:
def KNN():
    print "**************************************"
    tuned_parameters = {"n_neighbors": [5,7,9,11],
              "algorithm": ['auto','ball_tree','kd_tree']}
    
    clf = KNeighborsClassifier(n_neighbors=7)
    gs = GridSearchCV(clf, tuned_parameters, cv=5)
    gs.fit(X_train, Y_train)

    print("Best parameters set found on development set:")
    print(gs.best_params_)

    clf = KNeighborsClassifier(**gs.best_params_)
    #clfboost = AdaBoostClassifier(clf, n_estimators=50, learning_rate=1.0)
    #clfboost.fit(X_train, Y_train)
    
    clf = clf.fit(X_train, Y_train)
    y_pred = clf.predict(X_test)
    print '======================================'
    print 'KNN'
    classificationevaluation(Y_test, y_pred)
    print '======================================'
    print 'adaboost'
    #classificationevaluation(Y_test, clfboost.predict(X_test))

In [14]:
KNN()

**************************************
Best parameters set found on development set:
{'n_neighbors': 11, 'algorithm': 'auto'}
KNN
ar is:  0.492957746479
er is:  0.507042253521
Type I error rate:  0.17907444668
Type II error rate:  0.327967806841
adaboost
